# Modèles Statiques pour la gestion énergétique des bâtiments
Le développement de modèles statiques pour la gestion énergétique des bâtiments représente une opportunité prometteuse pour améliorer la durabilité environnementale et économique des bâtiments, et contribuer à la transition énergétique vers un avenir plus propre et plus durable. 

## Probleme
### Description du bâtiment
La maison considérée est un T4 de type rural d’environ 108m2 de surface habitable est composée comme suit : un hall d’entrée faisant office de pièce commune, trois chambres séparées, la maison est réalisée sur un seul niveau avec un volume de 324 m3 (Figure 1):  
- $L = 6, \, l = 3\,  \mathrm{m}$, longueurs,
- $H = 3 \, \mathrm{m}$, hauteur.
- $w = 0.20 \, \mathrm{m}$, largeur des murs.

![fig](./figures/Vue_plan_maison.png)
> Figure 1. Vue en plan de l’habitation

### Composantes de l’enveloppe : 
La maison qui est présentée comme une construction qui respecte des techniques bioclimatiques, sa composition est décrite comme suit : 

#### Mur Extérieur :
- Béton cellulaire 400 : épaisseur w = 10cm
- Isolation laine de roche : épaisseur w = 10cm
#### Mur intérieur :
- Béton cellulaire 400 : épaisseur w = 20cm
#### Fenêtre simple vitrage:
- Vitre 6mm
#### Fenêtre double vitrage HR:
- Vitre 6mm
- Air 6mm
- Vitre 6mm

### Donnée météorologique : 
Données recueillies pour le mois de Janvier 2015 depuis le site [Climat.one.bulding](https://climate.onebuilding.org/WMO_Region_6_Europe/FRA_France/index.html):
- Localisation : Grenoble Alpes Isère, France.
- Longitude et latitude : {N 45° 21.83'} {E 5° 18.80'}
- Elévation : 386 m au-dessus du niveau de la mer.
- Précipitations liquides moyennes standard mensuelles 71mm
- La température extérieure moyenne mensuelle est  $T_o = 3 \, \mathrm{°C}$. 
- La Vitesse moyenne mensuelle standard du vent  est  $V = 3.4 \, \mathrm{m/s}$.
- Le rayonnement solaire absorbé par chaque mur est $E = 800 \, \mathrm{W/m^2}$.
- Les coefficients de chaleur convective entre les murs et l'air intérieur et extérieur sont  $h_i = 8 \, \mathrm{W/(m^2 \, K)}$ et $h_o = 25 \, \mathrm{W/(m^2 \, K)}$ respectivement.

### Caractéristiques thermodynamiques des matériaux : 

- Béton cellulaire : $\lambda = 0.16 \, \mathrm{W/(m \, K)}$

- Isolation : $\lambda = 0.044 \, \mathrm{W/(m \, K)}$

- Fenêtre simple vitrage: $U_s = 6 \, \mathrm{W/(m^2 \, K)}$

- Fenêtre double vitrage: $U_d = 1.5 \, \mathrm{W/(m^2 \, K)}$

- Porte intérieur : $U_i = 2.6 \, \mathrm{W/(m \, K)}$

- Porte extérieur : $U_e = 2.5 \, \mathrm{W/(m \, K)}$

In [3]:
import numpy as np

np.set_printoptions(precision=1)

# Donnée
# ====
# dimensions
L, l, H, Lf, Hpi, lpi, Hpe, lpe, w_ext, w_int, w_iso  = 6, 3, 3, 1.25, 2, 0.90, 2, 1.04, 0.10, 0.20, 0.10  # m

# propriétés thermophysiques
λ_beton = 0.16      # W/(m K) conductivité thermique du béton
λ_iso = 0.044      # W/(m K) conductivité thermique de l'isolant
U_s = 6             # W/(m2 K) conductivité thermique de la fenetre simple vitrage
U_d = 1.5           # W/(m2 K) conductivité thermique du la fenetre double vitrage
U_i = 2.6           # W/(m2 K) conductivité thermique de la porte interieur
U_e = 2.5           # W/(m2 K) conductivité thermique du la porte exterieur
ρ, c = 1.2, 1000    # kg/m3, J/(kg K) densité, chaleur spécifique air
hi, ho = 8, 25      # W/(m2 K) coefficients de convection entrée, sortie

# rayonnement solaire à ondes courtes absorbé par chaque mur
E = 800             # W/m2

# température extérieure
To = 3              # °C

# taux de ventilation (renouvellements d'air par heure)
ACH = 1             # volume/h

# position fenetre, fenetre ouverte = 2 , fenetre fermé = 0 , fenetre entreouverte = 1
fenetre = 2

## Modèle thermique 

Le débit volumétrique d'air circulant dans une seule chambre (que ce soit la chambre 1, 2 ou 3) résulte du taux de ventilation : 
$$\dot V = L·l·H·\frac{ACH}{3600 \, \mathrm{s}}$$

Le débit massique d'air entrant et sortant des pièces 1,2 et 3 est :
$$\dot m = \rho \dot V$$

In [4]:
V_dot = 3*(L * l * H * ACH) / 3600  # débit d'air volumétrique
m_dot = ρ * V_dot                   # débit d'air massique

### Conductivité thermique 

Dans le modèle, on considérera les conductances :
- $h \, S$ - convection, W/K.
- $\lambda / w \, S$ - conduction, W/K.
- $\dot m c$ - advection, W/K.
- $K_p$ - gain du contrôleur proportionnel, W/K.
- $U_s \, S_f$ - convection fenêtre simple vitrage, W/K.
- $U_d \, S_f$ - convection fenêtre double vitrage, W/K.
- $U \, S_p$ - convection portes, W/K.


Nous considérerons les conductances des murs extérieurs et intérieurs (Figure 2). Les murs extérieurs sont modélisés avec une conductance pour la convection extérieure (cyan sur la figure 2) et une conductance équivalente pour la conductance dans le mur et la convection intérieure (bleu sur la figure 2). Les murs intérieurs sont modélisés par une conductance équivalente pour la convection intérieure, la conduction et la convection intérieure de l'autre côté.

![fig](./figures/Vue_plan_maison_conductance.png)
> Figure 2. Parois formant des conductances thermiques dans le modèle.

### Réseau thermique

Le réseau thermique modélisant le bâtiment est un graphe orienté pondéré (Figure 3). Il est composé d'arêtes (ou branches) et de sommets (ou nœuds).

À un nœud de température $\theta_j$, sont connectées des branches $q_k, k = 0 ... n$ et des sources de flux de chaleur, $f_j$.

Ce graphe orienté pondéré est caractérisé par :
- $A$ - [Matrice d'incidence](https://fr.wikipedia.org/wiki/Matrice_d%27incidence);
- $G$ - Matrice diagonale de [conductance](https://fr.wikipedia.org/wiki/R%C3%A9sistance_(%C3%A9lectricit%C3%A9));
- $b$ - vecteur de [sources de température](https://fr.wikipedia.org/wiki/Source_de_tension);
- $f$ - vecteur de sources de flux.


La solution de ce problème a grandement été inspirée du travail de ([Ghiaus (2013)](https://hal.archives-ouvertes.fr/hal-03605823/document)):

$$ \theta = (A^TGA)^{-1}(A^TGb + f)$$
et
$$ q = G(-A\theta + b)$$

![fig](./figures/reseau_thermique_maisonv2.png)
> Figure 3. modèle de réseau thermique pour le bâtiment représenté aux figures 1 et 2.

Pour notre modèle, le graphe comporte 26 branches et 8 nœuds (Figure 3).

In [5]:
nq, nθ = 26, 8  # nombre de branches de flux et de nœuds de température

### Matrice d'incidence
La [Matrice d'incidence](https://fr.wikipedia.org/wiki/Matrice_d%27incidence) d'un graphe orienté montre la relation entre les arêtes (ou branches) $k$ et les sommets (ou nœuds) $j$ du graphe (Figure 3):

$$A_{k,j} = \begin{cases}\phantom{-}
0 & \text{si le flux de chaleur } q_k \text{ n'est pas connecte au noeud }  \theta_j \\ 
+1 & \text{si le flux de chaleur } q_k \text{ entre dans le noeud }  \theta_j\\ 
-1 & \text{si le flux de chaleur } q_k \text{ sort du noeud }  \theta_l 
\end{cases}$$

La matrice d'incidence est une matrice creuse qui a :
- le nombre de lignes égal à $n_q$, le nombre de flux de chaleur $q_k$;
- le nombre de colonne égal à $n_{\theta}$, le nombre de nœuds de température $\theta_j$.

In [6]:
# Matrice d'incidence
# ================
A = np.zeros([nq, nθ])

# q0 ... q3 (branches cyan)
A[0, 0] = 1
A[1, 1] = 1
A[2, 5] = 1
A[3, 7] = 1

# q4 ... q7 (branches bleu)
A[4, 0], A[4, 3] = -1, 1
A[5, 1], A[5, 2] = -1, 1
A[6, 4], A[6, 5] = -1, 1
A[7, 7], A[7, 6] = -1, 1

# q8 ... q12 (branches jaune)
A[8, 2], A[8, 3] = -1, 1
A[9, 3], A[9, 4] = -1, 1
A[10, 6], A[10, 2] = -1, 1
A[11, 6], A[11, 3] = -1, 1
A[12, 6], A[12, 4] = -1, 1

# q13 ... q15 et q20 ... q25 (branches verte)
A[13, 3] = 1
A[14, 3], A[14, 6] = -1, 1
A[15, 6] = 1
A[20, 2] = 1
A[21, 2], A[21, 6] = -1, 1
A[22, 6] = 1
A[23, 4] = 1
A[24, 4], A[24, 6] = -1, 1
A[25, 6] = 1

# q16 ... q19 (branches rouge)
A[16, 2] = 1
A[17, 3] = 1
A[18, 4] = 1
A[19, 6] = 1

### Conductances
Le nombre de conductances correspond au nombre de flux qui est le nombre de lignes de la matrice d'incidence $A$

In [7]:
# Matrice de conductance
# ==================
G = np.zeros(A.shape[0])

#### Murs extérieurs : convection extérieure
Les conductances de [convection](https://fr.wikipedia.org/wiki/Convection) entre les murs et l'air extérieur (en cyan sur les figures 2 et 3) sont de la forme
$$G_i = h_o \, S_{o,k},\text{ avec } k = 0 ... 3$$
où $S_{o,k}$ est la surface du mur en contact avec l'air extérieur :
- $S_{o,0} = L \, H$
- $S_{o,1} = S_{o,2} = (L + l) H$
- $S_{o,3} = (2l + 3L + 2w)H$
- $S_{f} = Lf^2$  - Surface de la fenêtre
- $S_{pint} = Hpi * Lpi$ - Surface de la porte interieur
- $S_{pext} = Hpe * Lpe$ - Surface de la porte exterieur
- $S_{p4} = S_{o,0} - S_{f}$
- $S_{p5,6} = S_{o,1} - S_{f}$
- $S_{p7} = S_{o,3} - S_{pext}$

In [8]:
# G0 ... G3 (branches cyan) : convection extérieure
L4 = 2 * l + 3 * L + 2 * (w_iso + w_ext)      # longueur mur extérieur de la chambre 4
G[0] = ho * (L * H )
G[1] = ho * (L + l ) * H
G[2] = ho * (L + l ) * H
G[3] = ho * (L4 * H )

#### Murs extérieurs : conduction et convection intérieure
La [conductance équivalente](https://fr.wikipedia.org/wiki/Parall%C3%A8le_(%C3%A9lectricit%C3%A9)) pour [conduction](https://fr.wikipedia.org/wiki/Conduction_thermique) dans le mur et la [convection](https://fr.wikipedia.org/wiki/Convection) entre le mur et l'air intérieur (en bleu sur la figure 3) est :
$$G_k = \frac{1}{\frac{w}{\lambda} + \frac{1}{h_i}}S_{pk} + \frac{1}{\frac{1}{U} + \frac{1}{h_i}}S_{f},\text{ avec } k = 4 ... 7$$

In [9]:
# G4 ... G7 (branches bleues) : conduction, convection intérieure
S0_0 = L * H
S0_1 = S0_2 = (L + l) * H
S0_3 = (2*l + 3*L + 2*w_int) * H
S_f = Lf**2
S_pint = Hpi * lpi
Sp_ext = Hpe * lpe
Sp4 = S0_0 - S_f
Sp5 = Sp6 = S0_1 - S_f
Sp7 = S0_3 - Sp_ext

G[4] = (1 / (w_ext / λ_beton + w_iso / λ_iso + 1 / hi) * Sp4 ) + (1 / (1 / U_d + 1 / hi) * S_f )  # ici on a utilisé une fenetre double vitrage Ud, pour voir l'influance de la fenetre, on peut simplement mettre une simple vitre en changeant Ud par Us
G[5] = (1 / (w_ext / λ_beton + w_iso / λ_iso + 1 / hi) * Sp5 ) + (1 / (1 / U_d + 1 / hi) * S_f )
G[6] = (1 / (w_ext / λ_beton + w_iso / λ_iso + 1 / hi) * Sp6 ) + (1 / (1 / U_d + 1 / hi) * S_f )
G[7] = (1 / (w_ext / λ_beton + w_iso / λ_iso + 1 / hi) * Sp7 ) + (1 / (1 / U_e + 1 / hi) * Sp_ext )

#### Murs intérieurs : convection intérieure, conduction convection intérieure
La [conductance équivalente](https://fr.wikipedia.org/wiki/Parall%C3%A8le_(%C3%A9lectricit%C3%A9)) pour les murs intérieurs est formé par le (jaune sur les figures 2 et 3) :
- convention entre mur et air intérieur.
- conduction dans le mur.
- convection entre le mur et l'air intérieur.

Il est de la forme :
$$G_k = \frac{1}{\frac{2}{h_i} + \frac{w}{\lambda}}S_{i,j}, \text{ avec } k = 8 ... 12$$

où $S_{i,j}$ est la surface du mur en contact avec l'air intérieur :
- $S_{p8} = S_{p9} = l \, H$.
- $S_{p10} = S_{p11} = S_{p12} = S_{o,0} -  S_{pint}$.

In [10]:
# G8 ... G12 (branches jaunes) : murs intérieurs
# convection intérieure, conduction, convection intérieure
Sp8 = Sp9 = l * H
Sp10 = Sp11 = Sp12 = S0_0 - S_pint

G[8] = 1 / (2 / hi + w_int / λ_beton ) * Sp8
G[9] = 1 / (2 / hi + w_int / λ_beton ) * Sp9
G[10] = 1 / (2 / hi + w_int / λ_beton ) * Sp10 + 1 / (2 / hi + 1 / U_i ) * S_pint
G[11] = 1 / (2 / hi + w_int / λ_beton ) * Sp11 + 1 / (2 / hi + 1 / U_i ) * S_pint
G[12] = 1 / (2 / hi + w_int / λ_beton ) * Sp12 + 1 / (2 / hi + 1 / U_i ) * S_pint

#### Ventilation : transfert de chaleur par advection

La ventilation des pièces 1, 2, 3 et 4 transfère la chaleur à ces espaces par [advection](https://fr.wikipedia.org/wiki/Advection) (Figure 4). Le [flux de chaleur transféré par le fluide](https://fr.wikipedia.org/wiki/D%C3%A9bit_(physique)) est:
$$\dot H = \dot m c \theta$$
où :
- $\dot H$ est l'[enthalpie](https://fr.wikipedia.org/wiki/Enthalpie) taux transporté par l'air, W;
- $\dot m$ - débit massique de l'air, kg / s;
- $c$ - la [capacité thermique spécifique](https://fr.wikipedia.org/wiki/Capacit%C3%A9_thermique_massique) de l'air, J / (kg⋅K);
- $\theta$ - temperature de l'air, °C.

##### Débit d'air : extérieur → Chambre 1 → Chambre 4 → extérieur

Si le flux d'air va de l'extérieur vers la pièce 1, alors le flux de chaleur transféré de l'air extérieur à l'air intérieur de la pièce 1 est :
$$q_{20} = \dot m c T_o - \dot m c \theta_2$$
ou
$$q_{20} = G_{20}(T_o - \theta_2)$$
où $G_{20} = \dot m c$.

Le flux thermique transféré par advection de l'air de la chambre 1 vers la chambre 4 est :
$$q_{21a} = \dot m c \theta_2 - \dot m c \theta_6$$
ou
$$q_{21a} = G_{21}(\theta_2 - \theta_6)$$
où $G_{21} = \dot m c$.

##### Débit d'air : extérieur → Chambre 4 → Chambre 1 → extérieur

Si le flux d'air va de la Chambre 4 vers la Chambre 1 (Figure 4 b), alors le flux de chaleur transféré à l'air intérieur de la Chambre 1 est :
$$q_{21b} = \dot m c \theta_6 - \dot m c \theta_2$$
ou
$$q_{21b} = G_{21}(\theta_6 - \theta_2)$$
où $G_{21} = \dot m c$.

Le flux thermique transféré par convection de l'air extérieur vers la chambre 4 est de :
$$q_{22} = \dot m c T_o - \dot m c \theta_6$$
ou
$$q_{22} = G_{22}(T_o - \theta_6)$$
où $G_{22} = \dot m c.$

##### Flux de chaleur de la Chambre 1 à la Chambre 4

Si l'air passe de la chambre 1 à la chambre 4, alors le flux de chaleur est $q_{21a} \equiv q_{21}.$ Si l'air passe de la Chambre 4 à la Chambre 1, alors le flux de chaleur est $q_{21b} \equiv -q_{21}.$ Par conséquent, un seul flux de chaleur, $q_{21}$, est nécessaire dans le réseau thermique (Figure 3).

##### Modélisation du changement de direction du flux d'air

Si l'air circule dans le sens : extérieur → Chambre 1 → Chambre 4 → extérieur, alors la conductance $G_{22}$ est nul et les conductances $G_{20}$ et $G_{21}$ sont égaux à $\dot m c$ (Figure 3 et Figure 4a).

Si l'air circule dans le sens : extérieur → Chambre 4 → Chambre 1 → extérieur, alors la conductance $G_{20}$ est nul et les conductances $G_{21}$ et $G_{22}$ sont égaux à $\dot m c$ (Figure 3 et Figure 4b).

S'il n'y a pas de ventilation, les conductances $G_{20}$, $G_{21},$ et $G_{22}$ sont nuls.

##### Débit d'air : extérieur → Chambre 2 → Chambre 4 → extérieur

$$q_{13} = G_{13}(T_o - \theta_3)$$
où $G_{13} = \dot m c$.

Le flux thermique par advection est :

$$q_{14a} = G_{14}(\theta_3 - \theta_6)$$
où $G_{14} = \dot m c$.

##### Débit d'air : extérieur → Chambre 4 → Chambre 2 → extérieur

$$q_{14b} = G_{14}(\theta_6 - \theta_3)$$
où $G_{14} = \dot m c$.

Le flux thermique par convection est :

$$q_{15} = G_{15}(T_o - \theta_6)$$
où $G_{15} = \dot m c.$

##### Flux de chaleur de la Chambre 2 à la Chambre 4

Le comportement du flux de chaleur à travers les chambres 2 et 4 suit le même principe qu'au travers des chambres 1 et 4, on fait simplement l'analogie des résistances  de convection équivalentes.

##### Modélisation du changement de direction du flux d'air

Le comportement du flux d'air à travers les chambres 2 et 4 suit le même principe qu'au travers des chambres 1 et 4, on fait simplement l'analogie des résistances d'advection équivalentes.

##### Débit d'air : extérieur → Chambre 3 → Chambre 4 → extérieur

$$q_{23} = G_{23}(T_o - \theta_4)$$
où $G_{23} = \dot m c$.

Le flux thermique par advection est :

$$q_{24a} = G_{24}(\theta_4 - \theta_6)$$
où $G_{24} = \dot m c$.

##### Débit d'air : extérieur → Chambre 4 → Chambre 3 → extérieur

$$q_{24b} = G_{24}(\theta_6 - \theta_4)$$
où $G_{24} = \dot m c$.

Le flux thermique par convection est :

$$q_{25} = G_{25}(T_o - \theta_6)$$
où $G_{25} = \dot m c.$

##### Flux de chaleur de la Chambre 3 à la Chambre 4

Le comportement du flux de chaleur à travers les chambres 3 et 4 suit le même principe qu'au travers des chambres 1 et 4, on fait simplement l'analogie des résistances  de convection équivalentes.

##### Modélisation du changement de direction du flux d'air

Le comportement du flux d'air à travers les chambres 3 et 4 suit le même principe qu'au travers des chambres 1 et 4, on fait simplement l'analogie des résistances d'advection équivalentes.


![fig](./figures/08_vent_advection.svg)
> Figure 4. Advection de chaleur par ventilation. 
> - a) Débit d'air : extérieur → chambre 1, 2 ou 3 → chambre 4 → extérieur. 
> - b) Débit d'air : extérieur → chambre 4 → chambre 1, 2 ou 3 → extérieur.

In [11]:
# G13 ... G15 (branches vertes) : advection par ventilation

# si les fenêtres sont complètement fermé G13 = G20 = G23 = 0
if fenetre == 0 : 
    G[13:16] = np.zeros(3)
    G[20:26] = np.zeros(6)

# si les fenêtres sont à moitié ouverte
elif fenetre == 1 :
    G[15] = G[22] = G[25] = 0
    G[13:15] = G[20:22] = G[24:26] = 0.5 * m_dot * c

# si les fenêtres sont compltèment ouverte
elif fenetre == 2 : 
    G[15] = G[22] = G[25] = 0
    G[13:15] = G[20:22] = G[24:26] = m_dot * c

#### Régulateur (contrôleur) proportionnel

Le chauffage, la ventilation et la climatisation ([CVC](https://fr.wikipedia.org/wiki/Chauffage,_ventilation_et_climatisation)) le système maintient la température de l'air intérieur à [point de consigne](https://fr.wikipedia.org/wiki/Valeur_de_consigne) valeur. Par conséquent, le système CVC peut être modélisé comme un contrôleur proportionnel. (Figure 5a) pour lequel le flux de chaleur du système CVC est :

$$q = K_P (T_{sp} - \theta)$$

où:
- $K_P$ est le gain proportionnel
- $T_{sp}$ - température de consigne, °C;
- $\theta$ - température mesurée de l'air intérieur, °C.

Le flux de chaleur peut alors être exprimé comme (Figure 5b):
$$q = G (T_{sp} - \theta)$$
où $G \equiv K_p$.

Le contrôle de la température de l'air intérieur par le système CVC (Figure 5a) peut être modélisé par une source de température, $T_{sp}$, et une conductance, $G$ (Figure 5b).

![fig](./figures/08_P-controller.svg)
> Figure 5. Modèle d'un régulateur proportionnel dans un réseau thermique (exemple pour la Chambre 1). 
> - a) Principe d'un système de régulation de température utilisant un régulateur P. 
> - b) Modèle de système de contrôle de la température dans un réseau thermique.

Si les pièces sont en "free-running" (c'est-à-dire que les contrôleurs ne sont pas actifs), les gains proportionnels des contrôleurs sont nuls.

In [12]:
# G16 ... G19 (branches rouges) : gains des régulateurs proportionnels
G[16:20] = np.zeros(4)

### Vecteur de sources de température

Chaque bord (ou branche) du graphique a une [source de temperature](https://fr.wikipedia.org/wiki/Source_de_tension) qui n'est pas figuré s'il est identiquement nul (Figure 3). Le nombre de sources de température est égal au nombre de flux de chaleur, c'est-à-dire au nombre de lignes de la matrice d'incidence $A$.

In [13]:
# Vecteur de sources de température
# =============================
b = np.zeros(A.shape[0])

Les sources de température modélisant la température extérieure des murs sont sur les branches 0 ... 3.

In [14]:
b[0:4] = To         # branches cyan : température extérieure pour les murs

Les sources de température modélisant la température extérieure pour la ventilation se trouvent sur les branches 20 et 22 pour la Chambre 1, 13 et 15 pour la Chambre 2 et les branches 23 et 25 pour la Chambre 3.

In [15]:
b[[13, 15, 20, 22, 23, 25]] = To    # branches vertes : température extérieure pour la ventilation

Les sources de température modélisant les temperatures de [consigne](https://fr.wikipedia.org/wiki/Valeur_de_consigne) se trouvent sur les branches 16, ... 19. Les consignes pour les chambres sont :
- Chambre 1 est 20 °C.
- Chambre 2 est 22 °C.
- Chambre 3 est 20 °C.
- Chambre 4 est 24 °C.

In [16]:
b[[16, 17, 18, 19]] = 20, 22, 20, 24   # branches rouges : consignes des chambres 1, 2, 3 & 4

### Vecteur des sources de flux de chaleur

Dans notre modèle, les sources de flux de chaleur sont le rayonnement solaire absorbé par les murs extérieurs. Premièrement, nous considérerons que le rayonnement solaire est nul.

Le nombre de sources de flux de chaleur est égal au nombre de nœuds de température, c'est-à-dire au nombre de colonnes de la matrice d'incidence $A$.

In [17]:
# Vecteur des sources de flux
# =============================
f = np.zeros(A.shape[1])

### Les sorties

Dans ce problème, nous recherchons les températures de l'air intérieur des chambres (1, 2, 3 et 4) et les charges thermiques des pièces.

Les températures de l'air des pièces sont $\theta_2$, $\theta_3$, $\theta_4$, et $\theta_6$,

La charge thermique est le flux thermique nécessaire pour compenser tous les autres flux thermiques afin de maintenir la température de l'air à sa consigne. Par conséquent, les charges thermiques sont les flux de chaleur des régulateurs, c'est-à-dire $q_{16 ... 19}$.

In [18]:
# Index des sorties
# ==================
air_inter = [2, 3, 4, 6]   # nœuds de température de l'air intérieur
regulateur = range(16, 20)  # branches du régulateur

## Question 1 : les 4 chambres sont contrôlées

Considérant que les quatre pièces sont contrôlées par un système CVC aux températures de consigne 20 °C, 22 °C, 20 °C, 24 °C pour les chambres 1, 2, 3 et 4, respectivement et qu'il n'y a pas de rayonnement solaire sur les murs extérieurs, trouver les charges thermiques des quatre Chambres.

Pour un contrôleur proportionnel (regulateur) presque parfait, on considérera un très grand gain du contrôleur par rapport à la valeur maximale de la matrice de conductance.

In [19]:
print(f"Valeur maximale de conductance: {max(G):.0f} W/K")

Valeur maximale de conductance: 1830 W/K


In [20]:
b[regulateur] = 20, 20, 22, 18  # °C température de consigne des pièces
G[regulateur] = 1e9             # P-gain du contrôleur

θ = np.linalg.inv(A.T @ np.diag(G) @ A) @ (A.T @ np.diag(G) @ b + f)
q = np.diag(G) @ (-A @ θ + b)
print("1. Les 4 chambres contrôlées")
print("θ:", θ[air_inter], "°C")
print("q:", q[regulateur], "W")

1. Les 4 chambres contrôlées
θ: [20. 20. 22. 18.] °C
q: [1227.2 1165.2  476.9  675.2] W


## Question 2 : salles 2 & 4 en free-running

Considérons que :
- il y a un rayonnement solaire absorbé par les murs extérieurs, c'est-à-dire $f_k = E \, S_{o,j}, \text{ avec } k = 0, 1, 5, 7 \text{ et } j = 0...3;$
- comme auparavant, les pièces 1 et 3 sont contrôlées par un système CVC, c'est-à-dire $K_{P,j} \rightarrow \infty, \text{ avec } j=16, 18;$
- Les chambres 2 et 4 sont en "free-running", c'est-à-dire $K_{P,j} = 0, \text{ avec } j=17, 19.$ 

Trouver les charges thermiques des pièces 1 et 3 et les températures de l'air intérieur des pièces 2 et 4.

In [21]:
# Zone 2 & 4 en free-running ; radiation solaire; sans ventilation
G[[17, 19]] = 0     # gains du contrôleur pour les chambres 2 et 4

# radiation solaire
f[0] = E * Sp4
f[1] = E * Sp5
f[5] = E * Sp5
f[7] = E * Sp7

θ = np.linalg.inv(A.T @ np.diag(G) @ A) @ (A.T @ np.diag(G) @ b + f)
q = np.diag(G) @ (-A @ θ + b)
print("2. 2 & 4 free-run w/o ventilation")
print("θ:", θ[air_inter], "°C")
print("q:", q[regulateur], "W")

2. 2 & 4 free-run w/o ventilation
θ: [20.  12.7 22.  16.8] °C
q: [1040.6    0.   290.3    0. ] W


## Question 3 : ventilation de l'extérieur vers la chambre 1

Chambre 1 en ventilation avec un débit d'air $ACH$ (ACH est le nombre de renouvellements d'air par heure). L'air extérieur entre dans la chambre 1, puis il se déplace vers la chambre 4 et, à partir d'ici, il pénètre à l'extérieur.

Trouver les charges thermiques des pièces 1 et 3 et les températures de l'air intérieur des chambres 2 et 4.

Pour modéliser cette direction d'écoulement d'air, les conductances pour la ventilation sont :
$$G_{20} = G_{21} = \dot m c  ; G_{22} = 0 $$

In [24]:
# Zone 2 & 4 en free-running ; radiation solaire;
# Ventilation extérieure -> chambre 1 -> chambre 4 -> extérieure
ventilation = range(20, 23)
G[ventilation] = m_dot * c, m_dot * c, 0

θ = np.linalg.inv(A.T @ np.diag(G) @ A) @ (A.T @ np.diag(G) @ b + f)
q = np.diag(G) @ (-A @ θ + b)
print("3. 2 & 4 free-run, Ventilation extérieure -> 1 -> 4 -> extérieure")
print("θ:", θ[air_inter], "°C")
print("q:", q[regulateur], "W")

3. 2 & 4 free-run, Ventilation extérieure -> 1 -> 4 -> extérieure
θ: [20.  12.7 22.  16.8] °C
q: [1040.6    0.   290.3    0. ] W
